In [ ]:
# %pip install -q langchain langchain-groq pydantic faiss-cpu typing langchain_community langchain-huggingface

In [ ]:
%pip list

In [3]:
import os
import json
import csv
import logging
import pandas as pd
from typing import Dict, List, Optional
from langchain_groq.chat_models import ChatGroq
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import csv_loader
from pydantic import BaseModel, Field

c:\Users\Viraj Sawant\Desktop\SHLapp\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY= os.getenv("GROQ_API_KEY")

In [9]:
path = r"C:\\Users\\Viraj Sawant\\Desktop\\SHLapp\\app\\api\\data\\merged_data.csv"
data = pd.read_csv(path)

## **LangChain Model with Message History**







In [10]:

def get_test_type(row):
  test_types = []
  cols = ['Ability&Aptitude',
       'Biodata&SituationalJudgement', 'Competencies', 'Developemnt&360',
       'AssessmentExercies', 'Knowledge&Skills', 'Personality&Behavior',
       'Simulation']
  for col in cols:
    if row[col ] == 1:
      test_types.append(col)
  return ",".join(test_types)

def create_document(row):
  metadata = {
        "name": row["Assessment Name"],
        "url": row["Assessment URL"],
        "remote_support": row["Remote Testing Support"],
        "adaptive_support": row["Adaptive/IRT Support"],
        "duration": row["Duration"],
        "test_type": get_test_type(row)
    }

  return Document(page_content = row['Description'], metadata=metadata)

docs = [create_document(row) for _, row in data.iterrows()]

In [11]:
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

In [11]:
embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'} 
    )

In [15]:
vectorstore = FAISS.load_local(
        "C:\\Users\\Viraj Sawant\\Desktop\\SHLapp\\app\\api\\data\\vectorstorenew\\shl_faiss_embeddings", 
        embeddings,
        allow_dangerous_deserialization=True
    )

In [16]:
# embedding_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2",
#                                         model_kwargs=model_kwargs,
#                                         encode_kwargs=encode_kwargs)
# vectorstore = FAISS.from_documents(docs, embedding_model)

In [17]:
# vectorstore.save_local("/content/drive/MyDrive/__SHL__/vectorstorenew/shl_faiss_embeddings")

In [25]:
llm = ChatGroq(model="llama-3.3-70b-versatile",
               temperature = 0.3,
               api_key = GROQ_API_KEY)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002E421955650>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002E421957790>, model_name='llama-3.3-70b-versatile', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [26]:
QA_chain = RetrievalQA.from_llm(
    llm = llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 10}),
    return_source_documents=True,
    verbose = True
)

In [27]:
def format_response(response_docs):
    """Format the documents into a structured output."""
    results = []
    for doc in response_docs:
        meta = doc.metadata
        results.append({
            "Assessment Name": meta.get("name", "N/A"),
            "URL": meta.get("url", "N/A"),
            "Remote Support": meta.get("remote_support", "N/A"),
            "Adaptive Support": meta.get("adaptive_support", "N/A"),
            "Duration": meta.get("duration", "N/A"),
            "Test Type": meta.get("test_type", "N/A")
        })
    return results

def display_results(results):
    if not results:
        return "No matching tests found."
    df = pd.DataFrame(results)
    # For terminal or Jupyter
    print(df.to_markdown(index=False))
    return df

def ask_question(query: str):
    system_prompt = (
        "You are an assistant helping hiring managers choose the best SHL assessments. "
        "Based on the query, recommend the most relevant tests."
    )
    try:
        response = QA_chain.invoke({
            "query": system_prompt + "\n" + query  })

        docs = response.get("source_documents", [])
        formatted_results = format_response(docs)
        formatted_results = formatted_results[:10]

        return {
            "answer": response["result"],
            "recommendations": formatted_results
        }
    except Exception as e:
        logging.error(f"Error in ask_question: {str(e)}", exc_info=True)
        return {
            # "answer": f"Error: {str(e)}",
            "recommendations": []
        }

def get_recommendations(query):
    """Main entry function to get structured recommendations."""
    try:
        response = ask_question(query)
        print("\nAnswer from LLM:")
        # print(response["answer"])
        # print("\n Recommended Assessments Table:")
        return display_results(response["recommendations"])
    except Exception as e:
        logging.error(f"Error in get_recommendations: {str(e)}", exc_info=True)
        return f"Error: {str(e)}"


In [28]:
QUERY = "I am hiring for Java developers who can also collaborate effectively with my business teams. Looking \
for an assessment(s) that can be completed in 40 minutes."

In [29]:
if __name__ == "__main__":
    query = QUERY
    recommendations = get_recommendations(query)




> Entering new RetrievalQA chain...

> Finished chain.

Answer from LLM:
| Assessment Name                             | URL                                                                                                      | Remote Support   | Adaptive Support   |   Duration | Test Type                                                          |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------|:-----------------|:-------------------|-----------:|:-------------------------------------------------------------------|
| Verify Interactive Process Monitoring       | https://www.shl.com/solutions/products/product-catalog/view/verify-interactive-process-monitoring/       | Yes              | No                 |          0 | Ability&Aptitude                                                   |
| Assessment and Development Center Exercises | https://www.shl.com/solutions/products/product

In [ ]:
JOB_DESCRIPTION = """Role Overview:

We are seeking a highly skilled and motivated Software Development Engineer III (Full Stack) to join our growing team. As an SDE-3, you will play a pivotal role in designing, developing, and deploying robust, scalable, and high-performance full-stack applications. You will leverage your expertise in both front-end and back-end technologies to deliver exceptional user experiences and contribute to the architectural vision of our products.

Responsibilities:

Full Stack Development:
Design, develop, and maintain complex full-stack applications using modern technologies and frameworks.
Write clean, maintainable, and efficient code adhering to best practices and coding standards.
Develop and integrate RESTful APIs and microservices.
Implement responsive and intuitive user interfaces.
Architectural Contributions:
Participate in architectural discussions and contribute to the design and implementation of scalable and resilient systems.
Evaluate and recommend new technologies and tools to improve development efficiency and application performance.
Ensure adherence to architectural principles and design patterns.
Technical Leadership:
Mentor and guide junior developers, providing technical expertise and support.
Conduct code reviews and ensure code quality.
Drive technical initiatives and contribute to process improvements.
Lead small to medium-sized projects and features.
Collaboration and Communication:
Collaborate effectively with cross-functional teams, including product managers, designers, and QA engineers.
Communicate technical concepts clearly and concisely to both technical and non-technical stakeholders.
Participate in agile development processes, including sprint planning, daily stand-ups, and retrospectives.
Problem Solving and Debugging:
Troubleshoot and resolve complex technical issues.
Optimize application performance and ensure system stability.
Conduct thorough testing and debugging to ensure high-quality deliverables.
Required Skills and Qualifications:

Bachelor's or Master's degree in Computer Science, Software Engineering, or a related field.
Minimum [Number] years of professional experience in full-stack development.
Strong proficiency in back-end technologies:
[Specify your back-end technologies, e.g., Node.js, Python (Django/Flask), Java (Spring Boot), .NET Core].
Experience with database systems (e.g., PostgreSQL, SQL, MongoDB).
Strong proficiency in front-end technologies:
[Specify your front-end technologies, e.g., React, Angular, Vue.js, HTML5, CSS3, JavaScript/TypeScript].
Experience with state management libraries (e.g., Redux, Vuex).
Experience with cloud platforms (e.g., AWS, Azure, GCP).
Experience with version control systems (Git).
Strong understanding of software development principles, design patterns, and best practices.
Excellent problem-solving, debugging, and analytical skills.
Strong communication and collaboration skills.
Preferred Skills and Qualifications:

Experience with containerization and orchestration technologies (Docker, Kubernetes).
Experience with CI/CD pipelines (Jenkins, GitLab CI, CircleCI).
Knowledge of microservices architecture and distributed systems.
Experience with testing frameworks (e.g., Jest, Mocha, JUnit).
Experience with performance optimization and scalability.
Experience with serverless architecture.
Benefits:

Competitive salary and comprehensive benefits package.
Opportunities for professional growth and development.
Collaborative and innovative work environment.
Work-life balance and flexible work arrangements.
Access to cutting-edge technologies and tools.
Work for a globally recognized company.
To Apply:."""